# Trabajo Práctico: Modelos de Difusión para la Generación de Imágenes

## Introducción

Los modelos de difusión son una clase de modelos generativos que aprenden a generar datos de alta calidad revirtiendo un proceso de ruido progresivo. A medida que el modelo se entrena, aprende a deshacer el ruido para recuperar las imágenes originales. En este trabajo práctico, utilizaremos un modelo de difusión preentrenado para generar imágenes del conjunto de datos **CIFAR-10**, un dataset comúnmente usado en visión por computadora que contiene imágenes de 32x32 píxeles de 10 clases diferentes (como perros, gatos, automóviles, etc.).

A lo largo de este notebook, exploraremos cómo el modelo genera imágenes a partir de ruido y cómo diferentes parámetros afectan este proceso.

In [ ]:
# Instalamos las librerías necesarias
!pip install transformers diffusers imageio -q

^C


## Parte 1: Preparación del entorno

Primero, instalaremos y cargaremos las librerías necesarias. Utilizaremos **PyTorch** y el modelo preentrenado desde **Hugging Face**.

In [ ]:
import warnings
warnings.filterwarnings("ignore")
import torch
import time
import imageio
import numpy as np
from torchvision import transforms
from torchvision.utils import save_image, make_grid
from diffusers import DDPMPipeline
from matplotlib import pyplot as plt
from torchvision.datasets import CIFAR10
from torch.utils.data import DataLoader
import os

# Crear las carpetas './imgs' y './grid' si no existen
os.makedirs("./imgs", exist_ok=True)
os.makedirs("./grid", exist_ok=True)
os.makedirs("./gifs", exist_ok=True)

## Parte 2: Cargar el modelo preentrenado

Vamos a utilizar un modelo de difusión preentrenado en **CIFAR-10**. Hugging Face ofrece un pipeline ya entrenado que facilita la generación de imágenes.

El modelo pre-entrenado a utilizar se llama DDPM (Denoising Diffusion Probabilistic Models) de Jonathan Ho, Ajay Jain and Pieter Abbeel. Se puede encontrar la documentación en el siguiente link https://huggingface.co/docs/diffusers/api/pipelines/ddpm

In [ ]:
# Cargamos el modelo preentrenado DDPMPipeline para CIFAR-10
model_id = "google/ddpm-cifar10-32"
pipeline = DDPMPipeline.from_pretrained(model_id)
pipeline

## Parte 3: Generación de imágenes a partir de ruido

En esta sección, generaremos imágenes usando el modelo preentrenado. Cada imagen comenzará como ruido y el modelo lo "des-noiseará" gradualmente para producir una imagen reconocible.

El proceso de generación toma un número de **timesteps**, que son los pasos intermedios durante los cuales el modelo elimina progresivamente el ruido para obtener una imagen. Podemos ajustar el número de timesteps para ver cómo afecta a la calidad de la imagen.

### Ejercicio:
- ¿Qué sucede durante los “timesteps” en el proceso de generación de imágenes utilizando modelos de difusión?
- ¿Por qué comenzamos con una imagen completamente de ruido? ¿Qué papel juega el ruido en el proceso de generación de imágenes?
- Si la imagen original está completamente cubierta de ruido, ¿cómo es posible que el modelo aprenda a des-noisearla hasta obtener una imagen clara?

In [ ]:
def plot_imgs(imgs):
    # Mostrar las imágenes generadas
    plt.figure(figsize=(10, 10))
    for i, img in enumerate(imgs[:16]):
        plt.subplot(4, 4, i+1)
        plt.imshow(img)
        plt.axis("off")
    plt.show()


In [ ]:
# Generar imágenes
imgs = pipeline(num_inference_steps=1).images
plot_imgs(imgs)

## Parte 4: Explorando los timesteps.

El número de pasos de inferencia (`num_inference_steps`) refiere a los timesteps mencionados anteriormente.

### Ejercicio:
- Completar código.
- Modificar la cantidad de timesteps `num_inference_steps` para ver como varía el resultado (y e tiempo de inferencia) y analizar su impacto.
- ¿Cómo afecta el número de timesteps (pasos de inferencia) a la calidad de las imágenes generadas?
- ¿Cómo describirías visualmente las diferencias entre las imágenes generadas con 10 timesteps y 50 timesteps? ¿Qué esperas observar en cuanto a detalle y nitidez?

In [ ]:
# TODO: Generar 16 imágenes con distintos pasos de inferencia y plotearlas

## Parte 5: Manipulación del ruido

El ruido es una parte crítica del proceso de difusión. A partir de un ruido aleatorio, el modelo de difusión aprende a generar una imagen. Sin embargo, huggingface nos permite manipular la generación de este ruido pasandole una `seed`.

### Ejercicio
- Completar código.
- ¿Qué es una semilla (seed) y por qué es útil en el proceso de generación de ruido en modelos de difusión?
- ¿Cómo cambia la imagen generada si utilizas diferentes semillas para el ruido inicial? ¿Esperas que dos imágenes generadas con diferentes semillas sean completamente diferentes o tengan similitudes?
- Si utilizas la misma semilla para generar ruido dos veces, ¿esperas obtener la misma imagen al final del proceso de denoising? ¿Por qué?
- Modificar la semilla de generación de ruido para ver cómo varía el resultado y ver cómo impacta en la imagen generada.
- ¿Qué observas cuando cambias la semilla pero mantienes el resto de los parámetros del modelo constante?

In [ ]:
# TODO: Experimenta cambiando la semilla del generador de ruido (parámetro `generator` de la función `pipeline`)
generator = torch.manual_seed(...)

## Parte 6: Generación de imágenes en batch y visualización en grilla

Este proceso genera múltiples imágenes a la vez utilizando un **batch size** definido, y luego organiza esas imágenes en una grilla para visualizarlas.

### Proceso:
1. **Batch de ruido**: Se genera un batch de imágenes de ruido gaussiano utilizando un tamaño de batch específico (por ejemplo, `batch_size=8`).
2. **Generación en paralelo**: El pipeline genera varias imágenes en paralelo utilizando el batch de ruido.
3. **Grilla de imágenes**: Las imágenes generadas se organizan en una grilla utilizando la función `torchvision.utils.make_grid`, lo que permite visualizar varias imágenes en una única figura.

### Ejercicio:
- Completar código.
- Compare el tiempo de inferencia cuando genera en paralelo y secuencialmente (una por una).
- Compare el tiempo de inferencia cuando genera varias imagenes en paralelo y una sola.

In [ ]:
generated_images = ...

# Convertir las imágenes a tensores
image_tensors = [torch.tensor(np.array(img)).permute(2, 0, 1).unsqueeze(0) / 255.0 for img in generated_images]
image_tensors = torch.cat(image_tensors)

# Crear una grilla con las imágenes generadas
grid = make_grid(image_tensors, nrow=4)

# Guardar la grilla como una imagen PNG
save_image(grid, "grid/generated_images_grid.png")

# Visualizar la grilla de imágenes
plt.figure(figsize=(8, 8))
plt.imshow(grid.permute(1, 2, 0).cpu().numpy())
plt.axis('off')
plt.show()

## Parte 7: Generación paso a paso con el scheduler

En este proceso, utilizamos el **scheduler** del pipeline para gestionar la eliminación de ruido de una imagen inicial ruidosa, paso a paso.

### Proceso:

1. **Imagen inicial**: Comenzamos con una imagen de ruido gaussiano.
2. **Scheduler**: El **scheduler** controla cómo se elimina el ruido en cada paso, utilizando el modelo **UNet** para predecir y reducir el ruido progresivamente. El scheduler se encarga de realizar los pasos de denoising de manera estable.
3. **Captura de imágenes intermedias**: Guardamos las imágenes intermedias en intervalos específicos para visualizar cómo la imagen final emerge del ruido.
4. **Visualización**: Al final, se crea un GIF mostrando todo el proceso de generación, desde el ruido hasta la imagen final.

### Ejercicio:
- Completar código.
- ¿Qué observas al capturar imágenes intermedias durante el proceso de denoising? ¿Cómo cambia la imagen en cada paso?
- Explore con los parámetros `step_size` y `num_inference_steps`. ¿Para qué sirven?
- ¿Qué podemos aprender del GIF generado a partir de imágenes intermedias? ¿Qué observas sobre el proceso desde una imagen completamente ruidosa hasta una clara?
- ¿Por qué es útil visualizar el proceso de denoising paso a paso? ¿Cómo te ayuda esto a entender el funcionamiento del modelo de difusión?

In [ ]:
# Función para generar imágenes paso a paso con acceso al ruido intermedio
def generate_images_with_intermediate_steps(pipeline, noise, scheduler, num_inference_steps, step_size=1):
    images = []
    image = noise  # Imagen inicial con ruido

    for t in range(num_inference_steps):
        # Obtener el valor de timestep actual
        current_timestep = scheduler.timesteps[t]

        # El modelo predice el ruido en la imagen
        with torch.no_grad():
            model_output = pipeline.unet(image, current_timestep).sample  # Predecir el ruido

        # Realizar el paso del scheduler para actualizar la imagen con menos ruido
        image = scheduler.step(model_output, current_timestep, image).prev_sample

        # Guardar las imágenes en intervalos de step_size
        if t % step_size == 0:
            images.append(image.clone())  # Guardar imagen intermedia
            save_image(image, f"imgs/step_{t}.png")  # Guardar como PNG

    return images


In [ ]:
step_size = ...
num_inference_steps = ...
img_shape = (1, 3, 32, 32)

# Acceder al scheduler desde el pipeline
scheduler = pipeline.scheduler
scheduler.set_timesteps(num_inference_steps)

# Crear una imagen de ruido inicial (completamente ruidosa)
initial_noise = ...

# Generar las imágenes paso a paso
images = generate_images_with_intermediate_steps(pipeline, initial_noise, scheduler, num_inference_steps, step_size)

# Visualizar la última imagen generada
final_image = images[-1].squeeze().permute(1, 2, 0).cpu().numpy()
plt.imshow(final_image)
plt.axis('off')
plt.show()

image_paths = [f"imgs/step_{t}.png" for t in range(0, num_inference_steps, step_size)]
images_gif = [imageio.imread(path) for path in image_paths]
imageio.mimsave("gifs/step_by_step.gif", images_gif)

## Parte 8: Exploración del espacio latente entre dos imagenes

Vamos explorar el espacio latente entre dos imágenes. El objetivo es ver si podemos encontrar alguna relación entre las imagenes generadas por el movimiento del espacio latente entre dos imagenes.

### Proceso:
1. Generar dos estructuras con ruido diferente.
2. Interpolar entre ambas estructuras utilizando un valor de `alpha` que cambia de 0 a 1.
3. Pasar este ruido por el modelo de difusión.
4. Visualizar las imágenes generadas para observar la transición.

### Ejercicio:
- Completar código.
- ¿Qué es el espacio latente en un modelo de difusión y cómo se relaciona con las imágenes generadas?
- ¿Qué sucede cuando interpolamos entre dos estructuras de ruido diferentes utilizando un valor de alpha? ¿Qué esperarías ver en las imágenes generadas?
- Al visualizar las imágenes generadas por la interpolación, ¿observas alguna relación coherente entre las imágenes? ¿Las imágenes de la transición muestran características de ambas imágenes originales?

In [ ]:
# TODO: Completa la función `denoise_image` para realizar el denoising paso a paso. Hint: Basarse en la función `generate_images_with_intermediate_steps`
# Función para realizar el denoising paso a paso
# Dado un modelo UNet, una imagen ruidosa, un scheduler, el número de pasos de inferencia y el número de ruido que tenemos en las imágenes
#   esta función elimina el ruido de la imagen paso a paso.
def denoise_image(unet_model, noisy_image, scheduler, num_inference_steps, num_steps_added_noise=0):
    image = ...
    current_timestep = ...
    for t in range(num_inference_steps - num_inference_steps, num_inference_steps):
        # Predecir el ruido en la imagen actual

        # Actualizar la imagen eliminando el ruido predicho
        ...
    return image

In [ ]:
num_inference_steps = ...
scheduler = ...
# TODO: setear los timesteps del scheduler

# Crear dos puntos del espacio latente (imágenes con ruido)
image1_tensor = ...
image2_tensor = ...

# Interpolación entre las dos imágenes
steps = 10
interpolated_images = []
for alpha in np.linspace(0, 1, steps):
    # Interpolación de ruido entre las dos imágenes
    interpolated_noise = (1 - alpha) * image1_tensor + alpha * image2_tensor

    # Utilizamos el UNet del pipeline para predecir la imagen desde el ruido interpolado
    interpolated_image = denoise_image(pipeline.unet, interpolated_noise, scheduler, num_inference_steps, num_inference_steps)

    # Guardamos la imagen generada
    interpolated_images.append(interpolated_image.cpu().detach())

In [ ]:
# Visualización de las imágenes interpoladas
plt.figure(figsize=(15, 3))
for i, img_tensor in enumerate(interpolated_images):
    img = img_tensor.squeeze().permute(1, 2, 0).cpu().numpy()
    plt.subplot(1, steps, i + 1)
    plt.imshow(img)
    plt.axis('off')
plt.show()

# Crear el GIF de la interpolación
images_for_gif = []
for img_tensor in interpolated_images:
    img = img_tensor.squeeze().permute(1, 2, 0).cpu().numpy()
    img_uint8 = (img * 255).astype(np.uint8)  # Convertir a formato uint8 para el GIF
    images_for_gif.append(img_uint8)

# Guardar el GIF
imageio.mimsave('gifs/interpolation.gif', images_for_gif, duration=0.5)

## Parte 9: Añadir ruido controlado a una imagen real y eliminarlo

Vamos a tomar una imagen real del dataset, añadirle una cantidad controlada de ruido y luego intentaremos eliminar ese ruido utilizando el pipeline de difusión. Este proceso nos permitirá observar cómo el modelo reconstruye la imagen eliminando el ruido añadido.

### Pasos:
1. Seleccionar una imagen real del dataset CIFAR-10.
2. Añadir ruido gaussiano controlado a la imagen.
3. Usar el pipeline de difusión para eliminar el ruido y reconstruir la imagen original.

Este ejercicio ayuda a entender cómo los modelos de difusión pueden realizar la tarea de denoising en imágenes.

### Ejercicio:
- Completar código.
- ¿Qué es el ruido gaussiano y por qué lo añadimos a una imagen real en este ejercicio?
- ¿Cómo afecta la cantidad de ruido añadido a la imagen al proceso de denoising? ¿Qué esperarías que ocurriera si añades mucho ruido en comparación con poco?
- Si la cantidad de ruido añadido es muy alta, ¿crees que el modelo podrá eliminar todo el ruido y restaurar completamente la imagen original? ¿Por qué?
- ¿En qué escenarios prácticos crees que se podría aplicar este proceso de denoising en imágenes reales? ¿Qué aplicaciones puedes imaginar para esta técnica?

In [ ]:
# Cargar el dataset CIFAR-10 y seleccionar una imagen
transform = transforms.Compose([transforms.ToTensor()])
dataset = CIFAR10(root="./data", train=True, download=True, transform=transform)
dataloader = DataLoader(dataset, batch_size=1, shuffle=True)
image, _ = next(iter(dataloader))  # Tomar una imagen del dataset

# Acceder al UNet y el scheduler del pipeline
unet_model = pipeline.unet
scheduler = pipeline.scheduler

In [ ]:
# Configurar el scheduler con el número de pasos
num_inference_steps = ...
num_steps_added_noise = ...
scheduler.set_timesteps(num_inference_steps)

# Añadir ruido en el último paso del scheduler
timestep = scheduler.timesteps[-num_steps_added_noise]  # Tomar el último paso de la programación de ruido
with torch.no_grad():
    noisy_image = scheduler.add_noise(image, torch.randn_like(image), timestep)  # Añadir ruido

# Aplicar el proceso de denoising a la imagen ruidosa
denoised_image = ...

# Visualización de la imagen original, la imagen con ruido y la imagen denoised
plt.figure(figsize=(15, 5))

plt.subplot(1, 3, 1)
plt.imshow(image.squeeze().permute(1, 2, 0).cpu().numpy())
plt.title("Imagen original")
plt.axis('off')

plt.subplot(1, 3, 2)
plt.imshow(noisy_image.squeeze().permute(1, 2, 0).cpu().numpy())
plt.title("Imagen con ruido añadido")
plt.axis('off')

plt.subplot(1, 3, 3)
plt.imshow(denoised_image.squeeze().permute(1, 2, 0).cpu().numpy())
plt.title("Imagen denoised")
plt.axis('off')

plt.show()